
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement du changement climat climatique ou l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, produit par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif du pojet à l'IFÉ ENS de Lyon](https://contrib-tremplin.ens-lyon.fr/) et une copie se trouve dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. 

Les ressources  du projet **peuvent être utilisées dans tout autre environnement compatible**, notamment tous les cahiers d'exercices peuvent être exécutés sur toute machine disposant d'un python3  et des bibliothèques jupyter, jupyterlab, numpy, netcdf4. 

Les données _pré-traitées_ utilisées ci-dessous sont **accessibles  en ligne** sur le [serveur de données géolocalisées](http://geoloc-tremplin.ens-lyon.fr/climato-data/) `opendap` du projet tremplin.

![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche des enseignements autour du changement climatique : mitigation et adaptation au changement
# _Phase 2 : Création d'un jeu de données_

Ce cahier d'exercices `ipython` propose une méthode d'exploration d'un jeu de données issues des simulations climatiques de Météofrance à partir d'une série de fichiers issus de [DRIAS](http://www.drias-climat.fr/), concertis par le cahier d'exercice _"Phase 1 de cette série"_. Le lot utilisé  est issu de trois séries de modélisations  RCP 2.6 - 4.5 - 8.5 CNRM sur une grille 10 x 10 centrée sur la ville de Lyon. Le travail a été effectué avec la température maximale, la température minimale  et la pluviométrie (soit neuf fichiers source). Les données extraites de  [DRIAS](http://www.drias-climat.fr/) peuvent contenir d'autres variables non abordées ici mais les programmes ci-dessous peuvent aisément être transposés aux varables souhaitées.  

Cette seconde partie propose une méhode pour qu'un enseignant puisse extraire les données dont il a besoin d'une série de fichiers commandés sur le site [DRIAS](http://www.drias-climat.fr/), convertis et mis en ligne sur le [site du plojet](http://geoloc-tremplin.ens-lyon.fr/climato-data/). Ce site regroupe tous les échantillons produits par le projet et toutes les contributions fournies librement en suivant la procédure de la Phase1. 

Tous les cahiers de programmes sont libres et sous licence `creative commons` Ils sont accessibles sur la [forge du projet](https://contrib-tremplin.ens-lyon.fr/forge/). Ce cahier permet de produire un jeu de données Température/Pluviométrie autour de Lyon qui sera utilisé pour visualiser des courbes et des cartes en Phase3. Il peut être aisément modifié pour exploiter d'autres jeux de données du [site](http://geoloc-tremplin.ens-lyon.fr/climato-data/). Les cahiers d'exercices `ipython` créés peuvent être déposés et partagés sur la [forge du projet](https://contrib-tremplin.ens-lyon.fr/forge/)

Ce cahier manipule des données multidimensionnelles il est directement opérationnel et peut être simplement exécuté dans jupyter toutefois il doit être réservé à des étudiants avancés si on souhaite en analyser le code. Ce cahier utilise les données d'un carré 10 x 10 noeuds de la grille ALADIN centré sur la ville de Lyon. Le format d'entrée et de sortie est celui utilisé par les labos de climatologie, d'océanographie : `netCDF`.  Cet outil requiert l'installation des outils `netCDF4` et `numpy`

## Préparation de l'environnement et aperçu du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [1]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, datetime, os

Importation des données de températures maximales depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise et intégration dans un fichier pour le traitement, puis affichage de la description du contenu, de la liste des variables.

L'exemple choisi ici a été réalisé avec une grille de 10 x 10 noeuds centrés sur la ville de Lyon, pour obtenir un jeu de données depuis le site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) se reporter au manuel numérique réalisé par E. Le Jan et C. Larose dans le cadre du projet "Climat et Météo Tremplin pour l'enseignement des sciences". 

Le bloc de code ci-dessous est fourni à des fins de _"nettoyage et information"_ il n'est pas nécessaire de l'exécuter pour atteindre le résultat il permet d'obtenir les informations sur le fichier manipulé et enlève d'éventuelles scories d'exécutionq précédentes.

In [2]:
# Attention Avant de recharger les fichiers locaux depuis  les sources en ligne, 
# si des fichiers locaux sont OUVERTS
# il est nécessaire de les  fermer pour éviter des erreurs de données
# Dans tous les autres  situations
# L'exécution de ce bloc est inutile.
# Close Datasets if necessary
test1 = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
test2 = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
test3 = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
print('Variables disponibles :',test1.variables.keys()) # get all variable names
print('Variables disponibles :',test2.variables.keys()) # get all variable names
print('Variables disponibles :',test3.variables.keys()) # get all variable names
try: os.remove('tMax_26_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_45_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_85_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMin_26_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMin_45_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMin_85_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
test1.close()
test2.close()
test3.close()

Variables disponibles : odict_keys(['i', 'j', 'time', 'tasmax', 'lat', 'lon', 'x', 'y'])
Variables disponibles : odict_keys(['i', 'j', 'time', 'tasmin', 'lat', 'lon', 'x', 'y'])
Variables disponibles : odict_keys(['i', 'j', 'time', 'rstr', 'lat', 'lon', 'x', 'y'])


## Récupération des données 

Les 3 blocs de code suivants permettent de récupérer les données  de la variable choisie de puis le serveur en ligne du projet. Les impressions sont facultatives, elles permettent de vérifier le bon déroulement de l'pération.

### Températures maximales

In [3]:
tMax_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
lyon_tmax_26 = tMax_26_Lyon.variables['tasmax']
print('Structure et taille du tableau exporté :\n\t',tMax_26_Lyon.variables['tasmax'].dimensions, lyon_tmax_26.shape)
#print('\nPremière ligne de données :\n',lyon_temp_26[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_26[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [4]:
tMax_45_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',casting='safe')
lyon_tmax_45 = tMax_45_Lyon.variables['tasmax']
print('Structure et taille du tableau exporté :\n\t',tMax_45_Lyon.variables['tasmax'].dimensions, lyon_tmax_45.shape)
#print('\nPremière ligne de données :\n',lyon_temp_45[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_45[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [5]:
tMax_85_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',casting='safe')
lyon_tmax_85 = tMax_85_Lyon.variables['tasmax']          
print('Structure et taille du tableau exporté :\n\t',tMax_85_Lyon.variables['tasmax'].dimensions, lyon_tmax_85.shape)
#print('\nPremière ligne de données :\n',lyon_temp_85[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_85[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


### Températures minimales

In [6]:
tMin_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
lyon_tmin_26 = tMin_26_Lyon.variables['tasmin']
print('Structure et taille du tableau exporté :\n\t',tMin_26_Lyon.variables['tasmin'].dimensions, lyon_tmin_26.shape)
#print('\nPremière ligne de données :\n',lyon_temp_26[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_26[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [7]:
tMin_45_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',casting='safe')
lyon_tmin_45 = tMin_45_Lyon.variables['tasmin']
print('Structure et taille du tableau exporté :\n\t',tMin_45_Lyon.variables['tasmin'].dimensions, lyon_tmin_45.shape)
#print('\nPremière ligne de données :\n',lyon_temp_45[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_45[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [8]:
tMin_85_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',casting='safe')
lyon_tmin_85 = tMin_85_Lyon.variables['tasmin']          
print('Structure et taille du tableau exporté :\n\t',tMin_85_Lyon.variables['tasmin'].dimensions, lyon_tmin_85.shape)
#print('\nPremière ligne de données :\n',lyon_temp_85[0,0,:])
#print('Dernière ligne de données :\n',lyon_temp_85[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


### Pluviométrie

In [9]:
rstr_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
lyon_rstr_26 = rstr_26_Lyon.variables['rstr']
print('Structure et taille du tableau exporté :\n\t',rstr_26_Lyon.variables['rstr'].dimensions, lyon_rstr_26.shape)
#print('\nPremière ligne de données :\n',lyon_rstr_26[0,0,:])
#print('Dernière ligne de données :\n',lyon_rstr_26[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [10]:
rstr_45_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',casting='safe')
lyon_rstr_45 = rstr_45_Lyon.variables['rstr']
print('Structure et taille du tableau exporté :\n\t',rstr_45_Lyon.variables['rstr'].dimensions, lyon_rstr_45.shape)
#print('\nPremière ligne de données :\n',lyon_rstr_45[0,0,:])
#print('Dernière ligne de données :\n',lyon_rstr_45[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


In [11]:
rstr_85_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',casting='safe')
lyon_rstr_85 = rstr_85_Lyon.variables['rstr']          
print('Structure et taille du tableau exporté :\n\t',rstr_85_Lyon.variables['rstr'].dimensions, lyon_rstr_85.shape)
#print('\nPremière ligne de données :\n',lyon_rstr_85[0,0,:])
#print('Dernière ligne de données :\n',lyon_rstr_85[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)


## Récupération des paramètres de l'extraction (dates, i-j Aladin, lat-lon, x-y) 

À partir de la liste des variables obtenue ci-dessus on renomme les jeux de données de chacune des variables qui seront exploitées apour effectuer les calculs et contrôle de la taille des échantillons. Les affichages proposés permettent de contrôler que les paramètres présents sont effectivement ceux qui sont attendus.

In [12]:
# On utilise ici le calcul 2.6 mais les dates sont identiques sur les 3 jeux de données
# une seule extraction est nécessaire
lyon_date = tMax_26_Lyon.variables['time']  
print('Taille du tableau de dates - ',lyon_date.shape)
print('Date de début de la simulation : ',nc.num2date(lyon_date[0],lyon_date.units).strftime("%c"))
print('Date de fin de la simulation : ',nc.num2date(lyon_date[-1],lyon_date.units).strftime("%c"))

Taille du tableau de dates -  (34698,)
Date de début de la simulation :  Sun Jan  1 00:00:00 2006
Date de fin de la simulation :  Fri Dec 31 00:00:00 2100


In [13]:
lyon_lat,lyon_lon = tMax_26_Lyon.variables['lat'], tMax_26_Lyon.variables['lon'] # latitude longitude
print('Emprise du projet en latititude-Longitude ;\n', lyon_lat[0][0],'#',lyon_lon[0][0],' :: ',lyon_lat[-1][-1],'#',lyon_lon[-1][-1])
lyon_x,lyon_y = tMax_26_Lyon.variables['x'], tMax_26_Lyon.variables['y']  # coordonnées métriques
print('Emprise du projet en x-y mètres ;\n', lyon_x[0],'#',lyon_y[0],' :: ',lyon_x[-1],'#',lyon_y[-1])
lyon_gridi,lyon_gridj = tMax_26_Lyon.variables['i'], tMax_26_Lyon.variables['j'] # coordonnées grille Aladin
print('Emprise du projet en noeuds ALADIN ;\n', lyon_gridi[0],'#',lyon_gridj[0],' :: ',lyon_gridi[-1],'#',lyon_gridj[-1])

Emprise du projet en latititude-Longitude ;
 45.423668 # 4.3302917  ::  46.050774 # 5.2839203
Emprise du projet en x-y mètres ;
 756000 # 2049000  ::  828000 # 2121000
Emprise du projet en noeuds ALADIN ;
 88.0 # 55.0  ::  97.0 # 64.0


## Création d'un fichier de sauvegarde des moyennes mensuelles

Il a été pris le parti d'aborder les données sous la forme de moyennes mensuelles entre 2006 et 2100, ces données pourront ensuite être exploitées pour afficher l'évolution des tendances climatiques sur des intervalles de temps choisis par l'utilisateur. Ce chapitre inclut les deux étapes de création du container puis de calcul des valeurs des moyennes.

Le format choisi est le même que celui utilisé par météofrance pour fournir les données [NETCDF](https://www.unidata.ucar.edu/software/netcdf/), ce format est complexe mais permet aisément de stocker des données multidimensionnelles et il est accompagné d'outils numériques permettant d'explorer efficacement cet espace multidimensionnel.

Le premier bloc **effece un fichier qui porterait le même nom** et crée un nouveau fichier vide pour accueillir les valeurs moyennes calculées pour chaque noeud de la grille  traitée.


In [14]:
try: os.remove('tmin_tmax-rstr_Lyon-26-45-85.nc')  # par sécurité efface le fichier portadatain = numpy.array(['foo','bar'],dtype='S3')nt ce nom ! attention aux pertes possibles
except OSError : pass
extractLyonTempYearMonth = nc.Dataset('tmin_tmax-rstr_Lyon-26-45-85.nc',mode='w',format='NETCDF4') 

## Construction du jeu de paramètres : année mois latitude longitude x y

Définition et affectation des variables où sont copiées les paramètres de la grille et où seront stockés les résultats des calculs. Les affichages permettent de vérifier la validité des données utilisées.


In [15]:
# tableau du nom des mois
listmonth = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'All'])
# Tableau du nombre de jours par mois année ordinaire
lenMonthA =[31,28,31,30,31,30,31,31,30,31,30,31]
# Tableau du nombre de jours par mois année bissextile
lenMonthB =[31,29,31,30,31,30,31,31,30,31,30,31]
# Conversion en nombre entier des années extrêmes
firstyear = int(nc.num2date(lyon_date[0],lyon_date.units).strftime("%Y"))
lastyear = int(nc.num2date(lyon_date[-1],lyon_date.units).strftime("%Y"))
# Détermination de la taille des tableaux de calcul
sizemonths = len(listmonth)
twelvmonths = sizemonths - 1
sizeyears = lastyear - firstyear + 1 
sizegridi = lyon_gridi.shape[0]
sizegridj = lyon_gridj.shape[0]
print ("DateTime de départ  de l'étude : ", firstyear,
      "\nDateTime de fin  de l'étude : ", lastyear,
      "\nDurée de l'étude : ", sizeyears, 'ans',
      "\nNb de mois dans l'année : ", twelvmonths)
#
# Création des dimensions du tableau mois et année sont séparés pour disjoindre les traitements sur ces variables
#
extractLyonTempYearMonth.createDimension('i', sizegridi)     # latitude axis
extractLyonTempYearMonth.createDimension('j', sizegridj)    # longitude axis
extractLyonTempYearMonth.createDimension('month', sizemonths)    # month axis
extractLyonTempYearMonth.createDimension('year', sizeyears) # year axis
# Define two variables with the same names as dimensions,
# a conventional way to define "coordinate variables".
i = extractLyonTempYearMonth.createVariable('i', 'i4', ('i',))
i.long_name = 'cell index along first dimension'
j = extractLyonTempYearMonth.createVariable('j', 'i4', ('j',))
j.long_name = 'cell index along second dimension'
lat = extractLyonTempYearMonth.createVariable('lat', 'f4', ('j','i'))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lat.standard_name = 'latitude'
lat._CoordinateAxisType = 'Lat'
lon = extractLyonTempYearMonth.createVariable('lon', 'f4', ('j','i',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
lon.standard_name = 'longitude'
lon._CoordinateAxisType = 'Lon'
x = extractLyonTempYearMonth.createVariable('x', 'i4', ('i',))
x.long_name = 'cell index along first dimension'
y = extractLyonTempYearMonth.createVariable('y', 'i4', ('j',))
y.long_name = 'cell index along second dimension'
month = extractLyonTempYearMonth.createVariable('month','S3', ('month',))
month.units = 'month'
month.long_name = 'month_name'
month.standard_name = 'month_name'
year = extractLyonTempYearMonth.createVariable('year', 'u4', ('year',))
year.units = 'date'
year.long_name = 'year'
year.standard_name = 'year'
i[:] = lyon_gridi[:]
j[:] = lyon_gridj[:]
lat[:] = lyon_lat[:,:]
lon[:] = lyon_lon[:,:]
x[:] = lyon_x[:]
y[:] = lyon_y[:]
month[:] = listmonth

tmax_26 = extractLyonTempYearMonth.createVariable('tmax_26','f4',('year','month','j','i'),fill_value=1.e+20)
tmax_26.units = 'degree C' # degrees Celsius
tmax_26.standard_name = 'air_temperature_max_26' # this is a CF standard name
tmax_45 = extractLyonTempYearMonth.createVariable('tmax_45','f4',('year','month','j','i'),fill_value=1.e+20)
tmax_45.units = 'degree C' # degrees Celsius
tmax_45.standard_name = 'air_temperature_max_45' # this is a CF standard name
tmax_85 = extractLyonTempYearMonth.createVariable('tmax_85','f4',('year','month','j','i'),fill_value=1.e+20)
tmax_85.units = 'degree C' # degrees Celsius
tmax_85.standard_name = 'air_temperature_max_85' # this is a CF standard name
tmin_26 = extractLyonTempYearMonth.createVariable('tmin_26','f4',('year','month','j','i'),fill_value=1.e+20)
tmin_26.units = 'degree C' # degrees Celsius
tmin_26.standard_name = 'air_temperature_min_26' # this is a CF standard name
tmin_45 = extractLyonTempYearMonth.createVariable('tmin_45','f4',('year','month','j','i'),fill_value=1.e+20)
tmin_45.units = 'degree C' # degrees Celsius
tmin_45.standard_name = 'air_temperature_min_45' # this is a CF standard name
tmin_85 = extractLyonTempYearMonth.createVariable('tmin_85','f4',('year','month','j','i'),fill_value=1.e+20)
tmin_85.units = 'degree C' # degrees Celsius
tmin_85.standard_name = 'air_temperature_min_85' # this is a CF standard name
rstr_26 = extractLyonTempYearMonth.createVariable('rstr_26','f4',('year','month','j','i'),fill_value=1.e+20)
rstr_26.units = 'mm jr-1' # mm par jour
rstr_26.standard_name = 'largescale_rainfall_flux26' # this is a CF standard name
rstr_45 = extractLyonTempYearMonth.createVariable('rstr_45','f4',('year','month','j','i'),fill_value=1.e+20)
rstr_45.units = 'mm jr-1' # mm par jour
rstr_45.standard_name = 'largescale_rainfall_flux_45' # this is a CF standard name
rstr_85 = extractLyonTempYearMonth.createVariable('rstr_85','f4',('year','month','j','i'),fill_value=1.e+20)
rstr_85.units = 'mm jr-1' # mm par jour
rstr_85.standard_name = 'largescale_rainfall_flux_85' # this is a CF standard name
rstrc_26 = extractLyonTempYearMonth.createVariable('rstrc_26','f4',('year','month','j','i'),fill_value=1.e+20)
rstrc_26.units = 'mm jr-1' # mm par jour
rstrc_26.standard_name = 'largescale_cumRainfall_flux26' # this is a CF standard name
rstrc_45 = extractLyonTempYearMonth.createVariable('rstrc_45','f4',('year','month','j','i'),fill_value=1.e+20)
rstrc_45.units = 'mm jr-1' # mm par jour
rstrc_45.standard_name = 'largescale_cumRainfall_flux_45' # this is a CF standard name
rstrc_85 = extractLyonTempYearMonth.createVariable('rstrc_85','f4',('year','month','j','i'),fill_value=1.e+20)
rstrc_85.units = 'mm jr-1' # mm par jour
rstrc_85.standard_name = 'largescale_cumRainfall_flux_85' # this is a CF standard name
extractLyonTempYearMonth.title = 'Extrait TSMax par moyenne mensuelle de 2006 a 2100 Lyon et sa region'
extractLyonTempYearMonth.institution = 'ENS de Lyon'
extractLyonTempYearMonth.institute_id = 'IFE Institut Francais de l Education'
extractLyonTempYearMonth.project_id = 'Climat et meteo tremplin pour l enseignement des sciences'
extractLyonTempYearMonth.model_id = 'CNRM-ALADIN52'
extractLyonTempYearMonth.product = 'output derived from Meteofrance DRIAS data'
extractLyonTempYearMonth.contact = 'gerard.vidal@ens-lyon.fr'
extractLyonTempYearMonth.creation_date = str(datetime.datetime.now())
extractLyonTempYearMonth.driving_experiment_name = 'DRIAS2014'
extractLyonTempYearMonth.experiment = 'RCP2.6 RCP4.5 RCP8.5 '
extractLyonTempYearMonth.model = 'ALADIN-Climat'
extractLyonTempYearMonth.author = 'Gerard Vidal'
extractLyonTempYearMonth.comment = "Extraction des moyennes de la region Lyonnaise de 2006 a 2100 et changegement des variables"

DateTime de départ  de l'étude :  2006 
DateTime de fin  de l'étude :  2100 
Durée de l'étude :  95 ans 
Nb de mois dans l'année :  12


In [16]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])
#print('i :\n', i,i[:])
#print('j :\n', j,j[:])
#print('lon :\n', lon,lon[:,:])
#print('lat :\n', lat,lat[:,:])
#print('x :\n', x,x[:])
#print('y :\n', y,y[:])
#print('month :\n', month, '\n',datain,datain[:])
#print('i :\n',   extractLyonTempYearMonth.variables['i'][:])
#print('x :\n',   extractLyonTempYearMonth.variables['x'][:])
#print('lat :\n', extractLyonTempYearMonth.variables['lat'][:])
#print('month :', extractLyonTempYearMonth.variables['month'][:])

## Calcul principal des moyennes par mois pour chaque noeud et toutes les années


In [17]:
iteri = 0
iterj = 0
iteriFirst = iteri  
for iterj in  range(sizeyears) :
    year = (nc.num2date(lyon_date[iteri],lyon_date.units).strftime("%c"))
    #
    # Compteur permettant de suivre l'avancement du calcul et le fait que chaque année
    # commence bien le 1er janvier à 00h00 (suivi correct des mois et années bissextiles)
    # 
    print('\r',iterj,str(year), end = "")
    if (iterj % 4 == 2) :
        for p in range(len(lenMonthB)) :
            iteriLast = iteri + lenMonthB[p]
            # moyenne du mois année bissextile
            tmax_26[iterj,p,:,:] = np.mean(lyon_tmax_26[iteri:iteriLast,:,:] - 273, axis=0) 
            tmax_45[iterj,p,:,:] = np.mean(lyon_tmax_45[iteri:iteriLast,:,:] - 273, axis=0) 
            tmax_85[iterj,p,:,:] = np.mean(lyon_tmax_85[iteri:iteriLast,:,:] - 273, axis=0) 
            tmin_26[iterj,p,:,:] = np.mean(lyon_tmin_26[iteri:iteriLast,:,:] - 273, axis=0) 
            tmin_45[iterj,p,:,:] = np.mean(lyon_tmin_45[iteri:iteriLast,:,:] - 273, axis=0) 
            tmin_85[iterj,p,:,:] = np.mean(lyon_tmin_85[iteri:iteriLast,:,:] - 273, axis=0) 
            rstr_26[iterj,p,:,:] = np.mean(lyon_rstr_26[iteri:iteriLast,:,:], axis=0) 
            rstr_45[iterj,p,:,:] = np.mean(lyon_rstr_45[iteri:iteriLast,:,:], axis=0) 
            rstr_85[iterj,p,:,:] = np.mean(lyon_rstr_85[iteri:iteriLast,:,:], axis=0) 
            rstrc_26[iterj,p,:,:] = np.sum(lyon_rstr_26[iteri:iteriLast,:,:], axis=0)
            rstrc_45[iterj,p,:,:] = np.sum(lyon_rstr_45[iteri:iteriLast,:,:], axis=0) 
            rstrc_85[iterj,p,:,:] = np.sum(lyon_rstr_85[iteri:iteriLast,:,:], axis=0) 
            iteri = iteriLast
    else :
        for p in range(len(lenMonthA)) :
            iteriLast = iteri + lenMonthA[p]
            # moyenne du mois année ordinaire
            tmax_26[iterj,p,:,:] = np.mean(lyon_tmax_26[iteri:iteriLast,:,:] - 273, axis=0)
            tmax_45[iterj,p,:,:] = np.mean(lyon_tmax_45[iteri:iteriLast,:,:] - 273, axis=0) 
            tmax_85[iterj,p,:,:] = np.mean(lyon_tmax_85[iteri:iteriLast,:,:] - 273, axis=0)
            tmin_26[iterj,p,:,:] = np.mean(lyon_tmin_26[iteri:iteriLast,:,:] - 273, axis=0)
            tmin_45[iterj,p,:,:] = np.mean(lyon_tmin_45[iteri:iteriLast,:,:] - 273, axis=0) 
            tmin_85[iterj,p,:,:] = np.mean(lyon_tmin_85[iteri:iteriLast,:,:] - 273, axis=0)
            rstr_26[iterj,p,:,:] = np.mean(lyon_rstr_26[iteri:iteriLast,:,:], axis=0)
            rstr_45[iterj,p,:,:] = np.mean(lyon_rstr_45[iteri:iteriLast,:,:], axis=0)
            rstr_85[iterj,p,:,:] = np.mean(lyon_rstr_85[iteri:iteriLast,:,:], axis=0) 
            rstrc_26[iterj,p,:,:] = np.sum(lyon_rstr_26[iteri:iteriLast,:,:], axis=0)
            rstrc_45[iterj,p,:,:] = np.sum(lyon_rstr_45[iteri:iteriLast,:,:], axis=0)
            rstrc_85[iterj,p,:,:] = np.sum(lyon_rstr_85[iteri:iteriLast,:,:], axis=0) 
            iteri = iteriLast
    # moyenne de l'année
    # tmax_45[iterj,len(lenMonthA),:,:] = np.mean(lyon_tmax_45[iteriFirst:iteriLast,:,:] - 273, axis=0) 
    tmax_26[iterj,len(lenMonthA),:,:] = np.amax(tmax_26[iterj,0:twelvmonths,:,:], axis=0) 
    tmax_45[iterj,len(lenMonthA),:,:] = np.amax(tmax_45[iterj,0:twelvmonths,:,:], axis=0)
    tmax_85[iterj,len(lenMonthA),:,:] = np.amax(tmax_85[iterj,0:twelvmonths,:,:], axis=0) 
    tmin_26[iterj,len(lenMonthA),:,:] = np.amin(tmin_26[iterj,0:twelvmonths,:,:], axis=0) 
    tmin_45[iterj,len(lenMonthA),:,:] = np.amin(tmin_45[iterj,0:twelvmonths,:,:], axis=0) 
    tmin_85[iterj,len(lenMonthA),:,:] = np.amin(tmin_85[iterj,0:twelvmonths,:,:], axis=0) 
    rstr_26[iterj,len(lenMonthA),:,:] = np.mean(rstr_26[iterj,0:twelvmonths,:,:], axis=0) 
    rstr_45[iterj,len(lenMonthA),:,:] = np.mean(rstr_45[iterj,0:twelvmonths,:,:], axis=0) 
    rstr_85[iterj,len(lenMonthA),:,:] = np.mean(rstr_85[iterj,0:twelvmonths,:,:], axis=0) 
    rstrc_26[iterj,len(lenMonthA),:,:] = np.sum(rstrc_26[iterj,0:twelvmonths,:,:], axis=0) 
    rstrc_45[iterj,len(lenMonthA),:,:] = np.sum(rstrc_45[iterj,0:twelvmonths,:,:], axis=0) 
    rstrc_85[iterj,len(lenMonthA),:,:] = np.sum(rstrc_85[iterj,0:twelvmonths,:,:], axis=0) 
#
# extraction de  valeurs test sur une année et un mois donnés(aout 2050)
# permet de vérifier  l'existence d'anomalies flagrantes
# 
print('\nValeur moyenne 3 scénarios Tmax Lyon aout 2050 :\n',tmax_26[44,7,5,5],tmax_45[44,7,5,5],tmax_85[44,7,5,5])
print('\nValeur moyenne 3 scénarios Tmin Lyon aout 2050 :\n',tmin_26[44,7,5,5],tmin_45[44,7,5,5],tmin_85[44,7,5,5])
print('\nValeur moyenne 3 scénarios Pluvio. Lyon aout 2050 :\n',rstr_26[44,7,5,5],rstr_45[44,7,5,5],rstr_85[44,7,5,5])
print('\nValeur cumulée 3 scénarios Cumul Pluvio. Lyon aout 2050 :\n',rstrc_26[44,7,5,5],rstrc_45[44,7,5,5],rstrc_85[44,7,5,5])
print("\nValeur max sur l'année 2050 :\n",tmax_26[44,12,5,5],tmax_45[44,12,5,5],tmax_85[44,12,5,5])
print("\nValeur Tmin sur l'année 2050 :\n",tmin_26[44,12,5,5],tmin_45[44,12,5,5],tmin_85[44,12,5,5])
print("\nValeur moyenne des précipitations mensuellessur l'année 2050 :\n", rstr_26[44,12,5,5],rstr_45[44,12,5,5],rstr_85[44,12,5,5])  
print("\nValeur cumulée sur l'année 2050 :\n",   rstrc_26[44,12,5,5],rstrc_45[44,12,5,5],rstrc_85[44,12,5,5])    

 94 Fri Jan  1 00:00:00 2100
Valeur moyenne 3 scénarios Tmax Lyon aout 2050 :
 27.21591 27.036688 31.908592

Valeur moyenne 3 scénarios Tmin Lyon aout 2050 :
 16.542135 17.35004 21.048792

Valeur moyenne 3 scénarios Pluvio. Lyon aout 2050 :
 1.0227215 2.577805 0.52468824

Valeur cumulée 3 scénarios Cumul Pluvio. Lyon aout 2050 :
 31.704369 79.91196 16.265335

Valeur max sur l'année 2050 :
 27.21591 27.036688 31.908592

Valeur Tmin sur l'année 2050 :
 0.25848094 0.102317564 -0.5623496

Valeur moyenne des précipitations mensuellessur l'année 2050 :
 2.3966687 2.7075322 2.4695756

Valeur cumulée sur l'année 2050 :
 872.32086 989.03687 901.1476


Les affichages suivants permettent de vérifier  que les données obtenues correspondent au format attendu et présentent des valeurs cohérentes, plusieurs types de représentation sont proposés.

In [18]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])
#print('i :\n', i,i[:])
#print('lon :\n', lon,lon[:,:])
#print('x :\n', x,x[:])
#print('year :\n', year, year[:])
#print('month :', month, month[:])
#print(extractLyonTempYearMonth.variables['i'][:])
#print(extractLyonTempYearMonth.variables['month'][:])

In [19]:
extractLyonTempYearMonth.close()